In [1]:
%pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.4 MB 5.2 MB/s eta 0:00:03
   ------ --------------------------------- 2.6/16.4 MB 7.9 MB/s eta 0:00:02
   ----------- ---------------------------- 4.7/16.4 MB 9.3 MB/s eta 0:00:02
   ----------------- ---------------------- 7.1/16.4 MB 9.6 MB/s eta 0:00:01
   --------------------- ------------------ 8.9/16.4 MB 9.4 MB/s eta 0:00:01
   ------------------------- -------------- 10.5/16.4 MB 9.2 MB/s eta 0:00:01
   ------------------------------ --------- 12.3/16.4 MB 9.0 MB/s eta 0:00:01
   --------------------------------- ------ 13.9/16.4 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------  16.3/16.4 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 16.4/16.4 MB 8.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from mysql.connector import Error
import mysql.connector
import os  # Para lidar com caminhos de arquivos

In [ ]:
""" testar conexão com o banco de dados MySQL """

def test_mysql_connection(host, port, user, password, database):
    try:
        connection = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database
        )
        if connection.is_connected():
            print("Connection to MySQL database was successful!")
            print(f"Connected to database: {database}")
            print(f"Using user: {user}")
            connection.close()
        else:
            print("Failed to connect to MySQL database.")
    except Error as e:
        print(f"Error: {e}")


# Credenciais identificadas e utilizadas:
# Host: 'localhost' (para conexão do script na mesma máquina)
# Porta: 3306 (padrão do MySQL e confirmado)
# Usuário: 'tang' (o usuário criado para a aplicação no Docker)
# Senha: 'annabell' (a senha definida para o usuário 'tang')
# Banco de Dados: 'OurClassDB' (o banco de dados criado)
test_mysql_connection('localhost', 3307, 'tang', 'annabell', 'OurClassDB')

Connection to MySQL database was successful!
Connected to database: OurClassDB
Using user: tang


In [3]:
""" Mostrar todas as tabelas """
def get_all_table_data(host, port, user, password, database):
    """
    Conecta ao banco de dados MySQL, lista todas as tabelas, e então
    seleciona e imprime todos os dados de cada tabela.
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database
        )

        if connection.is_connected():
            cursor = connection.cursor()

            # 1. Listar todas as tabelas no banco de dados especificado
            print(
                f"Conectado ao banco de dados: '{database}' como usuário: '{user}'")
            cursor.execute("SHOW TABLES")
            tables = [table[0] for table in cursor.fetchall()]

            if not tables:
                print(
                    f"\nNenhuma tabela encontrada no banco de dados '{database}'.")
                return

            print(f"\n--- Tabelas em '{database}': {', '.join(tables)} ---")

            # 2. Selecionar e imprimir todos os dados de cada tabela
            for table_name in tables:
                print(f"\n--- Dados da tabela: '{table_name}' ---")
                try:
                    # Usar backticks para nomes de tabelas que possam ser palavras-chave
                    cursor.execute(f"SELECT * FROM `{table_name}`")
                    rows = cursor.fetchall()

                    if not rows:
                        print(
                            f"Nenhum dado encontrado na tabela '{table_name}'.")
                        continue

                    # Obter os nomes das colunas para melhor legibilidade
                    column_names = [i[0] for i in cursor.description]
                    print(f"Colunas: {', '.join(column_names)}")

                    for row in rows:
                        print(row)
                except Error as query_error:
                    print(
                        f"Erro ao selecionar dados da tabela '{table_name}': {query_error}")

        else:
            print("Falha ao conectar ao banco de dados MySQL.")

    except Error as e:
        print(f"Erro de conexão: {e}")
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("\nConexão MySQL encerrada.")


# --- Credenciais do seu banco de dados OurClassDB no Docker ---
db_host = 'localhost'      #
db_port = 3307             #
db_user = 'tang'           #
db_password = 'annabell'   #
db_name = 'OurClassDB'     #

# Chamar a função para obter e imprimir os dados de todas as tabelas
get_all_table_data(db_host, db_port, db_user, db_password, db_name)

Conectado ao banco de dados: 'OurClassDB' como usuário: 'tang'

--- Tabelas em 'OurClassDB': alunos, centers, disciplinas, horarios, laboratories, professores, salas, subjects, turmas ---

--- Dados da tabela: 'alunos' ---
Colunas: id, anoEntrada, curso, matricula, nome
(1, 2014, 'Ciência da Computação', '00114110001', 'Augusto Amaral Pereira')
(2, 2015, 'Ciência da Computação', '00115110002', 'Mateus Nunes Schulz')
(3, 2015, 'Ciência da Computação', '00115110003', 'Ralf Cruz Mateus')
(4, 2015, 'Ciência da Computação', '00115110004', 'Thiago Rodrigues')
(5, 2016, 'Ciência da Computação', '00116110005', 'Ian Pontes Louzada')
(6, 2016, 'Ciência da Computação', '00116110006', 'Jomacry Bruno Caetano')
(7, 2016, 'Ciência da Computação', '00116110007', 'Leonardo Nascimento Dias')
(8, 2016, 'Ciência da Computação', '00116110008', 'Matheus Dias')
(9, 2017, 'Ciência da Computação', '00117110009', 'Maria Luíza De Souza Stel')
(10, 2018, 'Ciência da Computação', '00118110010', 'Ariany Jasmyne Lop

In [2]:
""" Iniciar DB """


def execute_sql_file(host, port, user, password, db_to_connect_initially, sql_file_path):
    """
    Conecta ao banco de dados MySQL e executa comandos SQL de um arquivo.
    Cada comando é executado individualmente após ser dividido por ponto e vírgula.
    """
    connection = None
    try:
        # Conecta ao MySQL. Usamos 'root' e não especificamos um DB inicial (ou 'mysql')
        # para que o script SQL possa criar/selecionar o DB.
        connection = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=db_to_connect_initially  # Pode ser None ou 'mysql' para o usuário root
        )

        if connection.is_connected():
            cursor = connection.cursor()
            print(
                f"Conectado ao MySQL em {host}:{port} como usuário '{user}'.")

            # Verifica se o arquivo SQL existe
            if not os.path.exists(sql_file_path):
                raise FileNotFoundError(
                    f"Arquivo SQL não encontrado: '{sql_file_path}'")

            print(f"Lendo comandos SQL do arquivo: {sql_file_path}")
            with open(sql_file_path, 'r', encoding='utf8') as file:
                sql_commands_string = file.read()

            # Divide as instruções SQL por ponto e vírgula e filtra strings vazias.
            # Esta é uma divisão básica; para SQL muito complexo (com ; em strings/comentários),
            # um parser mais sofisticado seria necessário, mas funciona para dumps típicos.
            sql_statements = [
                s.strip() for s in sql_commands_string.split(';') if s.strip()
            ]

            print(
                f"\nExecutando {len(sql_statements)} comandos SQL no banco de dados...")

            for i, statement in enumerate(sql_statements):
                if statement:  # Garante que a instrução não esteja vazia
                    try:
                        # Opcional: Logar a instrução sendo executada para depuração
                        # print(f"  > Executando instrução {i+1}: {statement[:75]}...")
                        # Executa cada instrução individualmente
                        cursor.execute(statement)

                        # Confirma a execução (pode ser útil para DDL/DML)
                        # Ações como CREATE TABLE, INSERT, etc., não retornam um ResultSet diretamente.
                        # Se afetou linhas (INSERT, UPDATE, DELETE)
                        if cursor.rowcount:
                            print(
                                f"  > Instrução {i+1} executada (linhas afetadas: {cursor.rowcount})")
                        elif cursor.description:  # Se há colunas, é um SELECT
                            print(
                                f"  > Instrução {i+1} executada (resultado SELECT disponível)")
                        else:  # DDL (CREATE, ALTER, DROP)
                            print(
                                f"  > Instrução {i+1} executada (DDL/outros comandos)")

                    except Error as stmt_error:
                        # Loga a instrução específica que causou o erro
                        print(f"\n--- ERRO NA INSTRUÇÃO {i+1} ---")
                        # Mostra os primeiros 200 caracteres
                        print(f"Instrução: {statement[:200]}...")
                        print(f"Erro: {stmt_error}")
                        print("-----------------------------\n")
                        connection.rollback()  # Reverte todas as operações desde o último commit
                        raise  # Relança o erro para interromper a execução do script

            connection.commit()  # Confirma todas as alterações no final, após todas as instruções
            print(f"\nSQL do arquivo '{sql_file_path}' executado com sucesso!")

        else:
            print("Falha ao conectar ao banco de dados MySQL.")

    except FileNotFoundError:
        print(
            f"Erro: O arquivo SQL não foi encontrado no caminho especificado: '{sql_file_path}'")
    except Error as e:
        print(f"Erro de conexão ou execução de SQL: {e}")
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("Conexão MySQL encerrada.")


# --- Credenciais do seu banco de dados MySQL no Docker ---
# Usamos 'root' para garantir permissão de CREATE DATABASE
db_host = 'localhost'
db_port = 3307  # Confirme esta é a porta que seu container Docker está expondo no host
db_user = 'tang'  # Use o usuário 'root' para criação/gerenciamento de bancos de dados
db_password = 'annabell'  # Sua senha root

# Conecte-se a um banco de dados padrão (como 'mysql' ou None) inicialmente,
# pois o arquivo SQL já contém 'CREATE DATABASE OurClassDB' e 'USE OurClassDB'.
# Ou defina como None para não especificar um DB inicial.
db_initial_connect_name = 'OurClassDB'

# --- Caminho para o arquivo SQL ---
# Ajuste este caminho para o local exato do seu arquivo tableCreation.sql.
# Se o script Python estiver em 'B:\GitHub\UENF\timetabling-UENF\Files\Docker\backend\'
# E o arquivo SQL estiver em 'B:\GitHub\UENF\timetabling-UENF\Files\Docker\DB\StartingDB\tableCreation.sql'
# O caminho absoluto é a forma mais segura.
sql_file_path = r'B:/GitHub/UENF/timetabling-UENF/Files/Docker/DB/StartingDB/tableCreation.sql'


# --- Chamar a função para executar o arquivo SQL ---
execute_sql_file(db_host, db_port, db_user, db_password,
                 db_initial_connect_name, sql_file_path)

Conectado ao MySQL em localhost:3307 como usuário 'tang'.
Lendo comandos SQL do arquivo: B:/GitHub/UENF/timetabling-UENF/Files/Docker/DB/StartingDB/tableCreation.sql

Executando 115 comandos SQL no banco de dados...
  > Instrução 1 executada (linhas afetadas: 1)
  > Instrução 2 executada (DDL/outros comandos)
  > Instrução 3 executada (DDL/outros comandos)
  > Instrução 4 executada (DDL/outros comandos)
  > Instrução 5 executada (DDL/outros comandos)
  > Instrução 6 executada (DDL/outros comandos)
  > Instrução 7 executada (DDL/outros comandos)
  > Instrução 8 executada (DDL/outros comandos)
  > Instrução 9 executada (DDL/outros comandos)
  > Instrução 10 executada (DDL/outros comandos)
  > Instrução 11 executada (DDL/outros comandos)
  > Instrução 12 executada (DDL/outros comandos)
  > Instrução 13 executada (DDL/outros comandos)
  > Instrução 14 executada (DDL/outros comandos)
  > Instrução 15 executada (DDL/outros comandos)
  > Instrução 16 executada (DDL/outros comandos)
  > Instru